In [1]:
import geoengine as ge

In [2]:
ge.initialize("http://localhost:3030", admin_token="8aca8875-425a-4ef1-8ee6-cdfc62dd7525")

In [3]:
volumes = ge.volumes()
volumes

[Volume(name='nas_geo_data', path='/mnt/panq/dbs_research_data/geo_data/'),
 Volume(name='test_data', path='test_data/'),
 Volume(name='raid_geo_data', path='/mnt/data_raid/geo_data/')]

In [4]:
start_time = "1984-01-01T00:00:00Z"
end_time = "2023-01-02T00:00:00Z"

data_path_in_volume = "respect/ecmwf_era5"

named_variables = [
            ('Air Temperature', 'air_temperature', 'temperature', '°K'),
            ('Eastward Wind','eastward_wind','u_component_of_wind', 'X'),
            ('Northward Wind','northward_wind','v_component_of_wind', 'Y'),
            ('Specific Humidity','specific_humidity','specific_humidity', 'Z'),
            ('Relative Humidity','relative_humidity','relative_humidity', 'W'),
        ]

pressure_levels = [
            '300', '350', '400',
            '450', '500', '550',
            '600', '650', '700',
            '750', '775', '800',
            '825', '850', '875',
            '900', '925', '950',
            '975', '1000',
        ]

dataset_ids_air_temperature = {
    '300': "7c9cc95e-b3ed-461a-ae0e-21b2520d3391",
    '350': "3f76b90b-2eff-48fa-bb63-c6d5078392e7",
    '400': "105ad9f8-67f3-4cb0-b439-a92fb8443d22",
    '450': "37042fee-e7ff-40dd-9195-a04708e31a97",
    '500': "35e37da3-8c09-4471-988b-8a78cd4515b0",
    '550': "28a33e1a-9c7c-4d74-8b48-3941f4ac94e6",
    '600': "3fd40dd3-c89d-4ccc-a162-6ada9383735a",
    '650': "0a0acfdb-ac58-4d94-a07c-76e8db0c8c2f",
    '700': "88673023-2087-4282-acc8-8214bf2fb557",
    '750': "58fd9e61-1a4f-4b45-badd-c6a8f013de8a",
    '775': "68df526b-1dfc-49c4-aef7-035755ac491f",
    '800': "745398ab-c114-442d-af00-e6faeeb43f01",
    '825': "c6dd1a0f-7d8e-4435-bd59-4f6ea130f76e",
    '850': "8ce7ea1a-ec10-4048-b248-2570f304afe7",
    '875': "5ada9cec-1983-42a7-99d7-a41f2cb9963c",
    '900': "fd72b75f-7c67-4dbd-b82c-c313bfef2e18",
    '925': "ce768c83-b7a4-4b72-8a24-4a793eb53ccb",
    '950': "8835b0f1-6106-4ee2-b91b-5a136142fc58",
    '975': "d0135d9c-d1ce-495d-9d6e-41462ce93150",
    '1000': "06589b99-c9e4-4bb3-bde2-e425745c6255",
}

layer_ids_air_temperature = {
    '300': "dd3be565-c590-4155-9aea-662f9adfaa52",
    '350': "5ef4d541-ae7f-4ef9-85e5-8940cbed2faf",
    '400': "023c21a5-8d71-4582-83d6-75558361e043",
    '450': "b99bb974-56d8-42dc-ae56-ead202f7a469",
    '500': "c8c22bfb-d743-40ec-8c06-081b8cd3756c",
    '550': "df6a2f62-aa85-4167-8c91-4a353b21eb19",
    '600': "784fd7bf-479c-4d53-9210-21342f0d8079",
    '650': "bc34430a-4573-4be8-bdef-011e3215500d",
    '700': "2e902e7a-8bd6-4e39-9af0-294c9105ec15",
    '750': "e5699818-93d7-424e-b042-a4a325a46a97",
    '775': "cf214bfd-c40c-4491-87d3-1b71f8252199",
    '800': "32d1ba9e-1a72-4030-9b1b-2ca71f55ecaf",
    '825': "10ece546-882d-4f4e-8980-d9cea38269ff",
    '850': "8607429c-61fe-4db9-80d2-40b1177836c5",
    '875': "4dfd412a-80a4-439a-ab10-f85ba0ebae2e",
    '900': "ae861b63-66b3-43ce-8a60-f05884b4b770",
    '925': "6c649514-2b81-49af-8e48-1b6d81ac171f",
    '950': "89d5a6cc-8bb7-496d-b06f-19c97147a11e",
    '975': "efbbf7e6-b7d0-45fe-8d12-4077abbbc6eb",
    '1000': "0434f5d7-7c17-41b8-a789-49a0449a744e",
}


In [5]:
def generate_add_dataset(volume, data_path_in_volume, name, file_name_suffix, level, dataset_id, start_time, end_time):

  unscaled_colorizer = ge.Colorizer(
          breakpoints = [
            ge.ColorBreakpoint(-32766, (68, 1, 84, 255)),
            ge.ColorBreakpoint( -32307.5, (33, 145, 140, 255)),
            ge.ColorBreakpoint(-31849, (253, 231, 37, 255))
          ],
          no_data_color =(0, 0, 0, 0),
          default_color = (0, 0, 0, 0)
        )

  unscaled_symbology = ge.RasterSymbology(
        colorizer = unscaled_colorizer
      )

  era5_add_data_properties = ge.AddDataset(
      dataset_id = dataset_id,
      name = f'ECMWF ERA5 {name} Level {level}',
      description = f'ECMWF ERA5 {name} Level {level}',
      source_operator = "GdalSource",
      provenance = ge.Provenance(
        citation = "Hersbach, H., Bell, B., Berrisford, P., Biavati, G., Hor\u00e1nyi, A., Mu\u00f1oz Sabater, J., Nicolas, J., Peubey, C., Radu, R., Rozum, I., Schepers, D., Simmons, A., Soci, C., Dee, D., Th\u00e9paut, J-N. (2018): ERA5 hourly data on pressure levels from 1959 to present. Copernicus Climate Change Service (C3S) Climate Data Store (CDS). (Accessed on < DD-MMM-YYYY >), 10.24381/cds.bd0915c6",
        license = "https://cds.climate.copernicus.eu/api/v2/terms/static/licence-to-use-copernicus-products.pdf",
        uri = "https://doi.org/10.24381/cds.bd0915c6"
      ),
      symbology = unscaled_symbology,
  )

  era5_add_data_meta_data = ge.api.GdalMetaDataRegular({
      "type": "GdalMetaDataRegular",
      "resultDescriptor": ge.RasterResultDescriptor(data_type = "I16", spatial_reference = "EPSG:4326", measurement = ge.UnitlessMeasurement()).to_api_dict(),
      "params": ge.api.GdalDatasetParameters({
        "filePath": f'{data_path_in_volume}/%_YEAR_%/%_MONTH_%/%_START_DATE_%_{file_name_suffix}_{level}.tif',
        "rasterbandChannel": 1,
        "geoTransform": {
          "originCoordinate": {
            "x": -82.125,
            "y": 3.125
          },
          "xPixelSize": 0.25,
          "yPixelSize": -0.25
        },
        "width": 33,
        "height": 33,
        "fileNotFoundHandling": "NoData",
        "noDataValue": -32767
      }),
      "timePlaceholders": {
        "%_START_DATE_%": {
          "format": {
            'fmt': "%Y%m%d_%H%M",
            "has_tz": False,
            "has_time": True,
            },
          "reference": "start"
        },
        "%_YEAR_%": {
          "format": {
            'fmt':  "%Y",
            "has_tz": False,
            "has_time": True,
          },
          "reference": "start"
        },
        "%_MONTH_%": {
          "format": {
            'fmt': "%m",
            "has_tz": False,
            "has_time": True,
          },
          "reference": "start"
        }
      },
      "dataTime": {
        "start": start_time,
        "end": end_time
      },
      "step": {
        "granularity": "hours",
        "step": 1
      }
    })

  return (era5_add_data_properties, era5_add_data_meta_data) 

In [6]:
(name, file_name_suffix, _era5_variable, _measurement) = named_variables[0]
for level in pressure_levels:
  dataset_id = dataset_ids_air_temperature[level]
  (a,m) = generate_add_dataset(volumes[2], data_path_in_volume, name, file_name_suffix, level, dataset_id, start_time, end_time)
  print(a.to_api_dict())
  print(m)
  dataset_id = ge.add_public_raster_dataset(volumes[0].name, a, m)
  

{'id': '7c9cc95e-b3ed-461a-ae0e-21b2520d3391', 'name': 'ECMWF ERA5 Air Temperature Level 300', 'description': 'ECMWF ERA5 Air Temperature Level 300', 'sourceOperator': 'GdalSource', 'symbology': {'type': 'raster', 'colorizer': {'type': 'linearGradient', 'breakpoints': [{'value': -32766, 'color': (68, 1, 84, 255)}, {'value': -32307.5, 'color': (33, 145, 140, 255)}, {'value': -31849, 'color': (253, 231, 37, 255)}], 'noDataColor': (0, 0, 0, 0), 'defaultColor': (0, 0, 0, 0)}, 'opacity': 0.0}, 'provenance': {'citation': 'Hersbach, H., Bell, B., Berrisford, P., Biavati, G., Horányi, A., Muñoz Sabater, J., Nicolas, J., Peubey, C., Radu, R., Rozum, I., Schepers, D., Simmons, A., Soci, C., Dee, D., Thépaut, J-N. (2018): ERA5 hourly data on pressure levels from 1959 to present. Copernicus Climate Change Service (C3S) Climate Data Store (CDS). (Accessed on < DD-MMM-YYYY >), 10.24381/cds.bd0915c6', 'license': 'https://cds.climate.copernicus.eu/api/v2/terms/static/licence-to-use-copernicus-products

In [7]:
dataset_id = ge.add_public_raster_dataset(volumes[0].name, add_dataset_properties, meta_data)

dataset_id

NameError: name 'add_dataset_properties' is not defined